In [ ]:
!python -m pip install git+https://github.com/robfalck/OpenMDAO.git@math_subpackage

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/robfalck/OpenMDAO.git (to revision math_subpackage) to /tmp/pip-req-build-s4ab1ldm
  Running command git clone --filter=blob:none --quiet https://github.com/robfalck/OpenMDAO.git /tmp/pip-req-build-s4ab1ldm
  Running command git checkout -b math_subpackage --track origin/math_subpackage
  Switched to a new branch 'math_subpackage'
  Branch 'math_subpackage' set up to track remote branch 'math_subpackage' from 'origin'.
  Resolved https://github.com/robfalck/OpenMDAO.git to commit 1ac504fcd3aa42881137eba3046a6cd2cb66646b
  Preparing metadata (setup.py) ... done
  Created wheel for openmdao: filename=openmdao-3.26.1.dev0-py3-none-any.whl size=6126713 sha256=cdaf0e4513cb8b52893899a312bbc22172964ef53b4595e2e26ee3163f985207
  Stored in directory: /tmp/pip-ephem-wheel-cache-yi_3avpj/wheels/83/59/30/bf888efb14694b0dac57ae7b531996f98d9bfe812b69b6ef67
Successfully buil

In [3]:
def act_tanh(x, mu=1.0E-2, z=0., a=-1., b=1.):
    """
    Differentiable activation function based on the hyperbolic tangent.

    act_tanh can be used to approximate a step function from `a` to `b`, occurring at x=z.
    Smaller values of parameter `mu` more accurately represent a step function but the "sharpness" of the corners in the
    response may be more difficult for gradient-based approaches to resolve.

    Parameters
    ----------
    x : float or jnp.array
        The input at which the value of the activation function
        is to be computed.
    mu : float
        A shaping parameter which impacts the "abruptness" of
        the activation function. As this value approaches zero
        the response approaches that of a step function.
    z : float
        The value of the independent variable about which the
        activation response is centered.
    a : float
        The initial value that the input asymptotically approaches
        as x approaches negative infinity.
    b : float
        The final value that the input asymptotically approaches
        as x approaches positive infinity.

    Returns
    -------
    float or jnp.array
        The value of the activation response at the given input.
    """
    dy = b - a
    tanh_term = jnp.tanh((x - z) / mu)
    return 0.5 * dy * (1 + tanh_term) + a


In [4]:
from functools import partial

import jax
import jax.numpy as jnp

import openmdao.api as om
# from openmdao.math import act_tanh


class CountingComp(om.ExplicitComponent):
    
    def initialize(self):
        self.options.declare('vec_size', types=(int,))
        self.options.declare('threshold', types=(float,), default=0.0)
        self.options.declare('mu', types=(float,), default=0.01)
    
    def setup(self):
        n = self.options['vec_size']
        self.add_input('x', shape=(n,))
        self.add_output('count', shape=(1,))
        
        # The partials are a dense row in this case (1 row x N inputs)
        # There is no need to specify a sparsity pattern.
        self.declare_partials(of='count', wrt='x')

    @partial(jax.jit, static_argnums=(0,))
    def _compute_partials_jacfwd(self, x):
        deriv_func = jax.jacfwd(self.compute_primal, argnums=[0])
        dx, = deriv_func(x)
        return dx
    
    @partial(jax.jit, static_argnums=(0,))
    def _compute_partials_jacrev(self, x):
        deriv_func = jax.jacrev(self.compute_primal, argnums=[0])
        # Always returns a tuple
        dx, = deriv_func(x)
        return dx

    @partial(jax.jit, static_argnums=(0,))
    def _compute_partials_jvp(self, x):
        # Note that JVP is a poor choice here, since the jacobian is a row vector!
        # We have to call it once with each individual element in x set to 1.0
        # while all the others are zero in order to get a correct result!
        
        # jvp always returns the primal and the jvp
        # This will give incorrect results! There is "cross-talk" amongs the different
        # indices in the tangents.
        _, dx = jax.jvp(self.compute_primal,
                        primals=(x,),
                        tangents=(jnp.ones_like(x),))
        return dx

    # TODO: how to properly use vjp?
    @partial(jax.jit, static_argnums=(0,))
    def _compute_partials_vjp(self, x):
        # Note that JVP is a poor choice here, since the jacobian is a row vector!
        # We have to call it once with each individual element in x set to 1.0
        # while all the others are zero in order to get a correct result!
        
        # vjp always returns the primal and the vjp
        _, vjp_fun = jax.vjp(self.compute_primal, x)
        dx = vjp_fun(self.compute_primal(x))
        return dx
 
    # TODO: how to use jax.jit and get self.<attribute> into the function here? Wrapped function?
    @partial(jax.jit, static_argnums=(0,))
    def compute_primal(self, x):
        mu = self.options['mu']
        z = self.options['threshold']
        return jnp.sum(act_tanh(x, mu, z, 0.0, 1.0))
    
    def compute(self, inputs, outputs):
        z = self.options['threshold']
        x = inputs['x']
        mu = self.options['mu']
        
        outputs['count'] = self.compute_primal(*inputs.values())
        
    def compute_partials(self, inputs, partials):
        dx = self._compute_partials_jvp(*inputs.values())

        partials['count', 'x'] = dx

    def _tree_flatten(self):
        children = tuple()  # arrays / dynamic values
        aux_data = {'options': self.options}  # static values
        return (children, aux_data)

    @classmethod
    def _tree_unflatten(cls, aux_data, children):
        return cls(*children, **aux_data)

from jax import tree_util
tree_util.register_pytree_node(CountingComp,
                               CountingComp._tree_flatten,
                               CountingComp._tree_unflatten)


In [5]:
import numpy as np

N = 10

p = om.Problem()
p.model.add_subsystem('counter',
                      CountingComp(vec_size=N,threshold=0.5, mu=0.01),
                      promotes_inputs=['x'], promotes_outputs=['count'])
p.setup(force_alloc_complex=True)
p.set_val('x', np.linspace(0, 1, N))
p.run_model()
p.model.list_inputs(print_arrays=True)
p.model.list_outputs(print_arrays=True)

with np.printoptions(linewidth=1024):
    p.check_partials(method='cs', compact_print=False);

1 Input(s) in 'model'

varname  val                 
-------  --------------------
counter
  x      |1.87577145|        
         val:
         array([0.        , 0.11111111, 0.22222222, 0.33333333, 0.44444444,
                0.55555556, 0.66666667, 0.77777778, 0.88888889, 1.        ])


1 Explicit Output(s) in 'model'

varname  val 
-------  ----
counter
  count  [5.]


0 Implicit Output(s) in 'model'


---------------------------------
Component: CountingComp 'counter'
---------------------------------

  counter: 'count' wrt 'x'
    Analytic Magnitude: 1.890396e-02
          Fd Magnitude: 4.227054e-03 (cs:None)
    Absolute Error (Jan - Jfd) : 1.742859e-02 *

    Relative Error (Jan - Jfd) / Jfd : 4.123106e+00 *

    Raw Analytic Derivative (Jfor)
[[0.00597796 0.00597796 0.00597796 0.00597796 0.00597796 0.00597796 0.00597796 0.00597796 0.00597796 0.00597796]]

    Raw CS Derivative (Jcs)
[[7.44015195e-42 3.33096671e-32 1.49127858e-22 6.67647559e-13 2.98897836e-03 2.98897836e-03 6.6